# Dynamic Convolutional Neural Network
***
# Table of Contents
1.   [Imports](#Imports)
2.   [Dataset Object](#Dataset-Object)
3.   [Data Loading](#Data-Loading)
4.   [Model](#Model)
5.   [Setup](#Setup)
6.   [Training](#Training)
7.   [Testing](#Testing)

# Imports

The necessary libraries are imported at this stage.

* torch - Python pytorch library used to create and train the CNN
* numpy - Efficient data arrays
* pandas - Data applications
* sklearn - Provides a number of models, metrics and general functionality for machine learning.
* matplotlib - Provides plotting.

In [ ]:
import torch
import numpy as np
import pandas as pd
from torch.nn import Linear, Conv1d, MaxPool1d, Module, CrossEntropyLoss, Dropout
from torch.optim import Adam
from torch.utils.data import Dataset
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, fbeta_score, precision_score, recall_score, confusion_matrix
import itertools
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

# Dataset Object

Although not fully required it is helpful to create a custom torch.utils.data.Dataset when using the pytorch library.
This makes the splitting and feeding of data into the network easier.

## Encoding

A dict is defined 0-2 for the Dynamic activities in the dataset.

In [ ]:
activity_encode =   {
                        "WALKING": 0,
                        "STAIRS DN": 1,
                        "STAIRS UP": 2,
                        "SWIMMING" : 3,
                        "PUSH UP" : 4,
                        "JUMPING" : 5,
                        "DRIVING": 6,
                        "LAYING": 7,
                        "SITTING": 8,
                        "STANDING": 9
                    }

## Dataset

The object is defined as below for the UCI dataset. 3 functions need to be implemented **__init__**, **__len__** and
**__getitem__**.

In initialisation the data is split into X and y variables and turned into tensors.

The X variable has shape (row_count, 1, feature_count). 1 since there is only one channel of data.

The static features are removed.

In [ ]:
class UCI_Dynamic_Dataset(Dataset):
    """UCI dataset."""

    def __init__(self, csv_file, train=False):
        data = pd.read_csv(csv_file)

        data['Label'] = data['Label'].map(activity_encode)
        data, _ = [x for _, x in data.groupby(data['Label'] > 5)]
        if train:
            # shuffle data
            data = shuffle(data)
            # Get label value counts
            temp = data["Label"].value_counts()
            # Get the smallest value/ smallest feature count
            smallestV = int(temp.values[-1])
            # define counts
            label_counts = {0:[], 1:[], 2:[], 3:[], 4:[], 5:[]}
            # add each index to its label
            labels = data["Label"]
            for i, label in enumerate(labels):
                label_counts[label].append(i)
            # re define data dataframe by slicing indices
            new_df = pd.DataFrame(columns=data.columns)
            for label in label_counts:
                new_df = new_df.append(data.iloc[label_counts[label][:smallestV]])

            # return data as with even distribution
            data = new_df.copy()

        self.data_y = data['Label'].values
        data = pd.DataFrame(data.drop(['Label'],axis=1))
        self.data_x = np.array(data)
        # [batch, channels, features]
        self.data_x = self.data_x.reshape(len(self.data_x), 1, 589)
        self.data_x  = torch.from_numpy(self.data_x)
        self.data_y = self.data_y.astype(int)
        self.data_y = torch.from_numpy(self.data_y)

    def __len__(self):
        return len(self.data_x)

    def __getitem__(self, idx):
        return self.data_x[idx], self.data_y[idx]

Initialise the 3 datasets that will be used.

# 80 train 20 valid

In [ ]:
train_data = UCI_Dynamic_Dataset(csv_file='Our Dataset/train.csv', train=True)
valid_data = UCI_Dynamic_Dataset(csv_file='Our Dataset/valid.csv', train=True)
test_data = UCI_Dynamic_Dataset(csv_file='Our Dataset/test.csv')

def pie(csv_file, train=False):
    data = pd.read_csv(csv_file)
    data['Label'] = data['Label'].map(activity_encode)
    data, _ = [x for _, x in data.groupby(data['Label'] > 5)]
    if train:
        # shuffle data
        data = shuffle(data)
        # Get label value counts
        temp = data["Label"].value_counts()
        # Get the smallest value/ smallest feature count
        smallestV = int(temp.values[-1])
        # define counts
        label_counts = {0:[], 1:[], 2:[], 3:[], 4:[], 5:[]}
        # add each index to its label
        labels = data["Label"]
        for i, label in enumerate(labels):
            label_counts[label].append(i)
        # re define data dataframe by slicing indices
        new_df = pd.DataFrame(columns=data.columns)
        for label in label_counts:
            new_df = new_df.append(data.iloc[label_counts[label][:smallestV]])

        # return data as with even distribution
        data = new_df.copy()
    temp = data["Label"].value_counts()
    df = pd.DataFrame({'labels': temp.index,
                       'values': temp.values
                      })

    labels = df['labels']
    sizes = df['values']
    colors = ['yellowgreen', 'gold', 'lightskyblue', 'lightcoral', 'cyan','lightpink']
    patches, texts = plt.pie(sizes, colors=colors, shadow=True, startangle=90, pctdistance=1.1, labeldistance=1.2)
    plt.legend(patches, labels, loc="best")
    plt.axis('equal')
    plt.tight_layout()
    plt.show()
pie('Our Dataset/train.csv', train=True)
pie('Our Dataset/valid.csv', train=True)
pie('Our Dataset/test.csv')

# Data Loading

Using the DataLoader object load in the data, shuffling and assigning a batch size of 64.

In [ ]:
trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True)

# Model

The Dynamic CNN is implemented below, documentation is provided in the report.

In [ ]:
class DynamicCNN(Module):
    def __init__(self):
        super(DynamicCNN, self).__init__()

        self.conv1 = Conv1d(1, 100, kernel_size=tuple([3]))
        self.pool = MaxPool1d(kernel_size=tuple([3]))
        self.fc = Linear(19500, 6)
        self.dropout = Dropout(0.5)

    # Defining the forward pass
    def forward(self, x):
        # print(x.size()) #([64, 1, 589])
        x = self.pool(F.relu(self.conv1(x)))
        # print(x.size()) #([64, 100, 195])
        x = x.view(-1, 19500)
        # print(x.size()) #([64, 19500])
        x = self.dropout(F.log_softmax((self.fc(x))))
        # print(x.size()) #([64, 4])
        return x

# Setup

In this step we initialise the loss, optimizer and model objects. For this problem we used cross entropy loss and the
adam optimizer.

If found we use cuda GPU acceleration to make the process faster.

In [ ]:
criterion = CrossEntropyLoss()
device = "cuda" if torch.cuda.is_available() else "cpu"

model = DynamicCNN().double()
epochs = 11

step = 1
optimizer = Adam(model.parameters(), lr=0.00005)
# epochs = 5
#
# step = epochs
# optimizer = Adam(model.parameters(), lr=0.00005)
model.to(device)

# Training

In this step the model is trained. Validation is done so we can observe the progress.

In [ ]:
data_loaders = {}
data_loaders['train'] = trainloader
data_loaders['val'] = validloader
data_lengths = {"train": len(train_data), "val": len(valid_data)}
for epoch in range(epochs):
    if (epoch+1) % step == 0:
        print('Epoch {}/{}'.format(epoch, epochs - 1))
        print('-' * 10)

    # Each epoch has a training and validation phase
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train(True)  # Set model to training mode
        else:
            model.train(False)  # Set model to evaluate mode

        running_loss = 0.0

        # Iterate over data.
        for data in data_loaders[phase]:
            inputs, labels = data
            inputs, labels = inputs.to(device).double(), labels.to(device).long()
            # forward pass to get outputs
            output = model(inputs)
            # if labels == torch.tensor(1):
            #     print(labels)
            #     print(output)

            # calculate the loss between predicted and target keypoints
            loss = criterion(output, labels)

            # zero the parameter (weight) gradients
            optimizer.zero_grad()

            # backward + optimize only if in training phase
            if phase == 'train':
                loss.backward()
                # update the weights
                optimizer.step()

            # print loss statistics
            running_loss += loss.item()

        epoch_loss = running_loss / data_lengths[phase]
        if (epoch+1) % step == 0:
            print('{} Loss: {:.4f}'.format(phase, epoch_loss))

# Testing

Finally the model is tested on the test set.

In [ ]:
correct = 0
total = 0
model.cpu()
predictions = []
test_labels = []
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.double(), labels.long()
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        for p in predicted:
            # print(labels)
            # print(p)
            predictions.append(p)

        for l in labels:
            test_labels.append(l)


In [ ]:
#Confusion Matrix maps out the predicted label given to the data with the actual label
#Helps us check the rate of true/false positives and true/false negatives
#parameters are the true labels, and the predicted labels

# https://sklearn.org/auto_examples/model_selection/plot_confusion_matrix.html
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


cnf_matrix = confusion_matrix(test_labels, predictions)
np.set_printoptions(precision=2)

classes =    {
                        "WALKING": 0,
                        "STAIRS DN": 1,
                        "STAIRS UP": 2,
                        "SWIMMING" : 3,
                        "PUSH UP" : 4,
                        "JUMPING" : 5,
                    }
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes, normalize=True,title='Confusion matrix')


In [ ]:
print("Final accuracy score on the testing data: {:.4f}".format(accuracy_score(test_labels, predictions)))
print("Final F-score on the testing data: {:.4f}".format(fbeta_score(test_labels, predictions, beta = 0.5,average='weighted')))
print("Final precision score on the testing data: {:.4f}".format(precision_score(test_labels, predictions, average='weighted')))
print("Final recall score on the testing data: {:.4f}".format(recall_score(test_labels, predictions, average='weighted')))



